## Buckshot++: A Scalable, Noise-Tolerant Bagging Approach to Clustering. (Inspired by the [Buckshot Algorithm](https://pdfs.semanticscholar.org/1134/3448f8a817fa391e3a7897a95f975ad2873a.pdf).)

Here, we unveil a new algorithm that improves upon the k-means by dealing with the main shortcoming of k-means, namely, the need to predetermine the number of clusters (which we'll call K). Typically, K is found in the following manner: 
1. settle on some metric,
2. evaluate that metric at multiple values of K, 
3. use a greedy stopping rule to determine when to stop (typically the bend in an elbow curve).

There must be a better way. We detail the following 3 improvements that the Buckshot++ algorithm makes to k-means.   
1. Not all metrics are create equal. K-means doesn't prescribe which metric to use for finding K. We analyze below that some commonly implemented metrics are too inconsistent to be useful. Buckshot++ prescribes the silhouette score for finding K.
2. Another problem with k-means is that every single point is clustered--but what we really want is the pattern and not the noise. We show here a beautiful way to solve this problem -- a more elegant way than k-medoids and k-medians. 
3. Finally, the computational complexity of running k-means multiple times to find the best K can be prohibitive. We show a surprisingly simple alternative that scales better asymptotically. 

### The details of the Buckshot++ algorithm

**ALGORITHM**: Buckshot++ <br>
**INPUTS**: population of *N* vectors <br>
*B* := number of bootstrap samples <br>
*F* := max number of clusters to try <br>
*M* := cluster quality metric <br>
**OUTPUT**: the optimal *K* for kmeans

Take *B* bootstrap samples where each sample is of size 1/*B*.  
**for each** counter *k* from 2 to *F* **do** <br>
&emsp;&emsp;Compute kmeans with *k* centers. <br>
&emsp;&emsp;Compute the metric *M* on the clusters. <br>
Compute the centroid of all metrics vectors.  
Get argmax of the centroid vector.


### Explanation of the Buckshot++ algorithm
The main idea of the Buckshot++ algorithm is motivated by the [Buckshot algorithm](https://pdfs.semanticscholar.org/1134/3448f8a817fa391e3a7897a95f975ad2873a.pdf), which essentially finds cluster centers by performing hierarchical clustering on a sample and then performs k-means by taking those cluster centers as input. Here's why the Buckshot algorithm takes this approach:

In [34]:
tbl = DataFrame({'k-means': ['O(N * k * d * i)', 'random initial means; local minimum; outlier'],
                 'hierarchical': ['O(N^2 * logN)', 'outlier']}
                , index=['Computational Complexity', 'Sources of Instability'])
tbl

,k-means,hierarchical
Computational Complexity,O(N * k * d * i),O(N^2 * logN)
Sources of Instability,random initial means; local minimum; outlier,outlier


The big deal here is the high [computational complexity of hierarchical clustering](https://nlp.stanford.edu/IR-book/html/htmledition/time-complexity-of-hac-1.html), which is why the Buckshot algorithm performs hierarchical only on a sample. So in a nutshell, hierarchical is more deterministic/stable but less scalable than kmeans. 

Buckshot++'s key innovation lies in the step "Take B [bootstrap samples](http://www.stat.rutgers.edu/home/mxie/rcpapers/bootstrap.pdf) where each sample is of size 1/B." So Buckshot is doing hierarchical on a sample, while Buckshot++ is doing multiple kmeans on *bootstrap* samples. What's neat is that bootstrapping takes care of scalability and stability simultaneously. Doing kmeans many times can still finish sooner than doing hierarchical just once, as the time complexities above show. In terms of stability, bootstrapping is a great way to smooth out noise. In fact, that is exactly why **Bagging** (a.k.a. **B**ootstrap **Agg**regat**ing**) and [Random Forests](https://en.wikipedia.org/wiki/Random_forest#From_bagging_to_random_forests) work so well. 

### Let's see Buckshot++ in code:
Buckshot++ is implemented below. The data we use for illustration is the [Kaggle News Aggregator Dataset](https://www.kaggle.com/uciml/news-aggregator-dataset). In this data, there's a field that contains news headlines and another field that holds an ID indicating which story a headline belongs to.

In [35]:
from buckshotpp import Clusterings, plot_mult_samples

from numpy.random import choice
from pandas import read_csv, DataFrame
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score
import nltk; nltk.download('punkt')
import warnings; warnings.filterwarnings('ignore')

# Pass in settings to instantiate a Clusterings object called vecSpaceMod1:
vecSpaceMod = Clusterings({'file_loc': 'data/news_headlines.csv',
                           'tf_dampen': True,
                           'common_word_pct': 1,
                           'rare_word_pct': 1,
                           'dim_redu': False}
                         )
news_df = vecSpaceMod.get_file() # Read CSV input to data frame.
metrics_byK = vecSpaceMod.buckshot(news_df)        

ImportError: cannot import name 'plot_mult_samples'

### Interesting revelation from bootstrap
Each green curve is generated from a bootstrap sample, and the red curve is their average. Remember the sources of instability for k-means listed in the table above? Outlier is one. The concept of outlier has somewhat different meaning in the context of clustering. In supervised learning, an outlier is a rare observation that's far from other observations distance-wise. In clustering, a far away observation is its own well-separated cluster. So my interpretation is that "rare" is the operative word here and that outliers are singleton clusters that exert undue influence on the formation of other clusters. Look at how [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating) imparted a more stable estimate of the relationship between the number of clusters and the silhouette score in the graph below.

In [ ]:
plot_mult_samples(metrics_byK, 'silhouette')

#### Useful and not-so-useful internal clustering metrics currently out there.
The two internal clustering metrics implemented in scikit-learn are: the Silhouette Coefficient and the Calinski-Harabasz criterion. Comparing the Silhouette plotted above with the Calinski plotted below, it's clear that Silhouette is clearly better.

In [ ]:
plot_mult_samples(metrics_byK, 'calinski')

In [ ]:
X = vecSpaceMod.term_weight_matr(news_df.TITLE)
kmeans_fit = KMeans(20).fit(X)

#### Are the quality of the clusters good?
Taking a look at the documents and their corresponding "predictedCluster", the results certainly do impress!

In [ ]:
cluster_results = DataFrame({'predictedCluster': kmeans_fit.labels_,
                           'document': news_df.TITLE})
cluster_results.sort_values(by='predictedCluster', inplace=True)
cluster_results

#### Internal or External Clustering Metrics?
With the ground truth labels, we can evaluate Mutual Information, which is the most commonly used external metric. Looking at 0.67 Mutual Information compared with about 0.35 Silhouette, the relationship between the metrics is fairly consistent with other studies.

In [ ]:
mutual_info = adjusted_mutual_info_score(labels_true=news_df.STORY, labels_pred=kmeans_fit.labels_) 
print(mutual_info)

#### Buckshot++ Scales Better Than Buckshot.
The fact is that hierarchical clustering has a much higher order of time complexity than k-means. This means that, for sizable inputs, running k-means multiple times is still faster than running hierarchical clustering just once. The Buckshot algorithm runs hierarchical just once on a small sample in order to initialize cluster centers for k-means. Since O(N^2 * logN) grows really fast, the sample must be really small to make it work computationally. But a key critique of Buckshot is [failure to find the right structure with a small sample](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.136.7906&rep=rep1&type=pdf).  

### Our conclusion & the key advantages of Buckshot++
* **Extremely accurate** method of estimating the number of clusters (a clearly best Silhouette emerged every time, while typical elbow heuristic searches can hit or miss).
* **Highly scalable** (faster search for K achieved by using k-means rather than hierarchical; running k-means on subsample rather than everything).
* **Robust to noise** when used in conjunction with k-means++ (sampling with replacement lessens the chance of selecting an outlier in the bootstrap sample).